In [6]:
from xlrd import open_workbook

book = open_workbook('de/bo-heft-d-0-201506-xlsx.xlsx')
sheet=book.sheet_by_name("SVB - Tabelle I")

# read header values into the list    
keys = [sheet.cell(9, col_index).value for col_index in xrange(sheet.ncols)]
keys[0]="Berufsgruppen"
keys[1]="Ingesamt"
keys=keys[:4]


dict_list = []
for row_index in xrange(13, sheet.nrows-3):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value 
         for col_index in xrange(0,4)}
    dict_list.append(d)

In [7]:
for i, val in enumerate(dict_list):
    dict_list[i]["KldB"]=val["Berufsgruppen"].split(" ",1)[0]#int(val["Berufsgruppen"].split(" ",1)[0])
    dict_list[i]["Berufsgruppen"]=val["Berufsgruppen"].split(" ",1)[1]

In [8]:
#select only subsub groups
subsubgroups=[]
for i, val in enumerate(dict_list):
    if len(dict_list[i]["KldB"]) ==4:
        subsubgroups.append(dict_list[i])

In [9]:
import pandas as pd
import numpy as np

subsubgroups_df=pd.DataFrame.from_dict(subsubgroups)
subsubgroups_df[~subsubgroups_df[["Frauen","Ingesamt",u'M\xe4nner']].applymap(np.isreal).all(1)]

,Berufsgruppen,Frauen,Ingesamt,KldB,Männer
205,Berufe im Schornsteinbau,*,1553,3213,*
207,Berufe im Fassadenbau,*,1725,3215,*
209,Berufe im Bauwerksabbruch,*,668,3217,*
467,Staatsanwälte/-anwältinnen,*,*,7314,*
469,Berufe im Verfassungsschutz,*,*,7316,*
517,Tierheilpraktiker/innen,24,*,8153,*
519,Führung - Tiermedizin und Tierheilkunde,35,*,8159,*


In [10]:
#dell all 0s
df_all=subsubgroups_df[subsubgroups_df[["Frauen","Ingesamt",u'M\xe4nner']].applymap(np.isreal).all(1)]
print len(df_all)

689


In [11]:
df_all['Frauen_ratio'] =  df_all.Frauen*100/df_all.Ingesamt
df_all.to_csv("de/labourmarket_subgroups.csv", sep='\t', encoding='utf-8')


C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Read file with profession_names

In [12]:
book = open_workbook('de/Alphabetisches-Verzeichnis-Berufsbenennungen-Stand12032015.xls')
sheet=book.sheet_by_name("alphabet_Verz_Berufsb")

profession={}
for row_index in xrange(5, sheet.nrows-2):
    profession[sheet.cell(row_index, 0).value]=str(int(sheet.cell(row_index, 1).value))\
    if str(sheet.cell(row_index, 1).value)[0]!="0" else str(sheet.cell(row_index, 1).value)

#     

In [13]:
import re
k=0

g=0
x=0
profession_1={}
profession_2={}
profession_2_2={}
profession_3_0={}
for i in profession:
    if "(e/in)" in i:
        j=i.replace("(e/in)","e")
        x+=1
    elif "(er/in)" in i:
        j=i.replace("(er/in)","er")
    elif "e/r" in i:
        j=i.replace("e/r","er")
    elif "mann/-frau" in i:
        j=i.replace("mann/-frau","mann")
    elif "verk\xe4ufer/in".decode('ISO 8859-16') in i:
        j=i.replace("verk\xe4ufer/in".decode('ISO 8859-16'),"verk\xe4ufer".decode('ISO 8859-16'))
    elif "keramiker/in" in i:
        j=i.replace("keramiker/in","keramiker")
    elif "Medientechnologe/-technologin" in i:
        j=i.replace("Medientechnologe/-technologin","Medientechnologe")
    else:
        j=i.replace("er/in","er")
    j=re.sub("[*?\(\[].*?[\)\]]", "", j).strip() 
    if len(j.split("/"))>2:
        #print i
        k+=1
        profession_3_0[j.split("/")[0]]=profession[i]

    elif len(j.split("/"))==2:
        profession_2[j.split("/")[0].replace("(","").strip()]=profession[i]
        if j.split("/")[1][0].isupper():
            profession_2_2[re.sub("[*?\(\[].*?[\)\]]", "", j.split("/")[1]).strip()]=profession[i]
    else:
        g+=1
        profession_1[j]=profession[i]

In [14]:
print k,len(profession_3_0),len(profession_2),len(profession_1),g,x

327 186 4769 18790 20215 26


In [15]:
#before addine replace("er/in","er")
print k,len(profession_3_0),len(profession_2),len(profession_1),g,x

327 186 4769 18790 20215 26


In [16]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    

def find_appropriate_prof(page_m,words,profession,profession_2,profession_2_2,profession_1,profession_3_0):
    g=0
    l=0
    s=0
    y=0
    y1=0
    y2=0
    yy=0
    page_m_index={}
    other=[]
    for i in page_m:
        if i in profession.keys():
            page_m_index[i]=profession[i]
        elif i in profession_2.keys():
            page_m_index[i]=profession_2[i]
        elif i in profession_2_2.keys():
            page_m_index[i]=profession_2_2[i]
            l+=1
        elif i in profession_1.keys():
            page_m_index[i]=profession_1[i]
            s+=1
        elif i in profession_3_0.keys():
            page_m_index[i]=profession_3_0[i]
        elif i.replace(" f\xfcr ".decode('ISO 8859-16'), " - ").replace(" im ", " - ") in profession_1.keys():
            page_m_index[i]=profession_1[i.replace(" f\xfcr ".decode('ISO 8859-16'), " - ").replace(" im ", " - ")]
            y1+=1
        elif i.replace(" f\xfcr ".decode('ISO 8859-16'), " - ").replace(" im ", " - ") in profession_2.keys():
            y2+=1
        else:
            for prof in profession_1:
                if i in prof:
                    y+=1
                    print i,"=?",prof, profession_1[prof]
            for prof in profession_2:
                if i in prof:
                    yy+=1
                    print "==",i,"=?",prof, profession_2[prof]

            g+=1
            other.append(i)
            #print i

    print  l,g  ,s  ,y1,y2,y , yy
    return page_m_index,other


# Find code of profession for m_page

In [17]:
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
page_m=load_simple_json('de/wiki/m_page.json')

page_m_index,other=find_appropriate_prof(page_m,words,profession,profession_2,profession_2_2,profession_1,profession_3_0)

Techniker =? Techniker - Betonbau 32113
Techniker =? Techniker - Datenverarbeitungstechnik 26313
Techniker =? Techniker - Fertigungstechnik 27303
Techniker =? Techniker - Mess- und Regeltechnik 26123
Techniker =? Techniker - Kraftfahrzeugtechnik 25213
Techniker =? Techniker - Fahrzeugbau 25213
Techniker =? Techniker  - Agrartechnik 12103
Techniker =? Techniker - Personenverkehrssysteme 51503
Techniker =? Techniker - Automatisierungstechnik 26123
Techniker =? Techniker - Baubetrieb 31103
Techniker =? Techniker - Mühlenbau, Getreide- u. Futtermitteltechnik 25183
Techniker =? Techniker - Betriebswirtschaft 27303
Techniker =? Techniker - Markscheidetechnik 31213
Techniker =? Techniker - Verfahrenstechnik 41313
Techniker =? Techniker - Maschinenbautechnik 25103
Techniker =? Techniker - Verkehrsbau 31133
Techniker =? Techniker - Optik 21363
Techniker =? Techniker - Lebensmittelverfahrenstechnik 29203
Techniker =? Techniker - Automatisierte Produktion 26123
Techniker =? Techniker - Korrosions

In [18]:
page_m_index["Kettler"]='28122'
page_m_index["Euromaster"]='26314'
page_m_index["Energieelektroniker"]='26252'
page_m_index["Gehilfe"]='72302'
page_m_index["Krankenpfleger"]='81302'
page_m_index["Mitarbeiter"]='61123'
page_m_index["Geograph"]='42134'
page_m_index["Ordner"]='53112'
page_m_index[u'W\xe4chter']='53112'
page_m_index[u'F\xe4rber']='28101'
page_m_index["Religionslehrer"]='84424'
page_m_index[u'Stoffpr\xfcfer']='41322'
page_m_index["Reiniger"]='54101'


#we ommit Kaufmann,Fachkaufmann, Sachverständiger, Angestellter,
#as of being too broaden or representing only one person in the Labour Markt
other_new=set(other) - set(["Meister","Fachpraktiker","Verwalter",u'Sachverst\xe4ndiger',"Kaufmann","Helfer","Trainer",
                            "Vertreter","Facharbeiter",u'Angestellter',"Techniker",u'Fachkaufmann',"Werker",
                            "Fachleiter","Verfahrensmechaniker","Produktionshelfer",
                            "Energieelektroniker",u'Gehilfe',u'Krankenpfleger',u'Kettler',"Euromaster",
                           "Mitarbeiter","Geograph","Ordner",u'W\xe4chter',u'F\xe4rber',"Religionslehrer",u'Stoffpr\xfcfer',
                           "Reiniger"])
print len(other)
print len(other_new)

42
13


In [19]:
other_new

{u'Aufzugsmonteur',
 u'Fachangestellter f\xfcr Medien- und Informationsdienste',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (B\xe4ckerei)',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (Fleischerei)',
 u'Kraftfahrer',
 u'Masseur und medizinischer Bademeister',
 u'Online-Redakteur',
 u'Produktgestalter Textil',
 u'Produktpr\xfcfer Textil',
 u'Produktveredler Textil',
 u'Pr\xe4sident',
 u'Vorpolierer Schmuck- und Kleinger\xe4teherstellung',
 u'Werkgehilfe Schmuckwarenindustrie, Taschen- und Armbanduhren'}

In [20]:
manually_retrieved={"Aufzugsmonteur":'25132',
 u'Fachangestellter f\xfcr Medien- und Informationsdienste':'73332',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (B\xe4ckerei)':'62312',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (Fleischerei)':'62322',
 u'Kraftfahrer':'51311',
 u'Masseur und medizinischer Bademeister':'81712',
 u'Online-Redakteur':'92413',
 u'Produktgestalter Textil':'28112',
 u'Produktpr\xfcfer Textil':'28122',
 u'Produktveredler Textil':'28142',
 u'Pr\xe4sident':'84394',
 u'Vorpolierer Schmuck- und Kleinger\xe4teherstellung':'24222',
 u'Werkgehilfe Schmuckwarenindustrie, Taschen- und Armbanduhren':'71382'}
for i in manually_retrieved:
    page_m_index[i]=manually_retrieved[i]
len(page_m_index),len(page_m)

(886, 902)

In [21]:
print "We do not took into account:"
set(page_m)-set(page_m_index.keys())

We do not took into account:


{u'Angestellter',
 u'Facharbeiter',
 u'Fachkaufmann',
 u'Fachleiter',
 u'Fachpraktiker',
 u'Helfer',
 u'Kaufmann',
 u'Meister',
 u'Produktionshelfer',
 u'Sachverst\xe4ndiger',
 u'Techniker',
 u'Trainer',
 u'Verfahrensmechaniker',
 u'Vertreter',
 u'Verwalter',
 u'Werker'}

# Masculine_bias

In [22]:
page_m=load_simple_json('de/wiki/m_bias.json')
print len(page_m)
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
m_bias_index,other=find_appropriate_prof(page_m,words,profession,profession_2,profession_2_2,profession_1,profession_3_0)

812
Metallformer =? Metallformermeister 24193
Metallformer =? Metallformer u. Metallgießer 24142
Metallformer =? Metallformer- und Metallgießermeister 24193
Meister =? Meister - Schuhfertigung 28393
Meister =? Meister - Buchbinderische Weiterverarbeitung 23493
Meister =? Meister - Getreidewirtschaft 29293
Meister =? Meister - Fernmeldebau 26393
Meister =? Meister - Schmuckindustrie 93593
Meister =? Meister - Energieversorgung und -unterhalt 26293
Meister =? Meister - Modistinnenhandwerk 28293
Meister =? Meister - Walzwerktechnik 24193
Meister =? Meister - Kunststoffchemie 41393
Meister =? Meister - Seefischerei 11493
Meister =? Meister - Fotochemie 41393
Meister =? Meister - Montagebau 31193
Meister =? Meister - Elektroindustrie 26393
Meister =? Meister - Bootsbauerhandwerk 25293
Meister =? Meister - Schlosser- und Schmiedehandwerk 24493
Meister =? Meister - Brunnenbauerhandwerk 32293
Meister =? Meister - Flugzeuginstandhaltung 25293
Meister =? Meister - Landmaschinenmechanikerhandwerk

In [33]:
m_bias_index["Metallformer"]='24142'

m_bias_index["Geograph"]='42134'
m_bias_index["Medienkaufmann"]='92302'
m_bias_index["Gehilfe"]='72302'
m_bias_index[u'Stoffpr\xfcfer']='41322'

m_bias_index[u'Medienberater']='92113'

m_bias_index[u'F\xe4rber']='28101'
m_bias_index["Krankenpfleger"]='81302'
m_bias_index["Mitarbeiter"]='61123'
m_bias_index[u'W\xe4chter']='53112'

m_bias_index["Energieelektroniker"]='26252'
m_bias_index["Religionslehrer"]='84424'


#we ommit Kaufmann,Fachkaufmann, Sachverständiger, Angestellter,
#as of being too broaden or representing only one person in the Labour Markt
other_new=set(other) - set(["Meister","Fachpraktiker",u'Sachverst\xe4ndiger',"Helfer","Trainer",
                            "Facharbeiter","Verfahrensmechaniker",u'Angestellter',"Techniker",u'Fachkaufmann',
                            "Kaufmann","Fachleiter","Produktionshelfer",
                            "Geograph",u'Medienberater',"Medienkaufmann",u'Gehilfe',u'Stoffpr\xfcfer',"Mitarbeiter",
                            u'W\xe4chter',u'Krankenpfleger',u'F\xe4rber',"Religionslehrer","Energieelektroniker",
                            "Metallformer"])
print len(other)
print len(other_new)

40
15


In [34]:
other_new

{u'Aufzugsmonteur',
 u'Fachangestellter f\xfcr Medien- und Informationsdienste',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (B\xe4ckerei)',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (Fleischerei)',
 u'Film-Regisseur',
 u'Kettengoldschmied',
 u'Kraftfahrer',
 u'Masseur und medizinischer Bademeister',
 u'Online-Redakteur',
 u'Produktgestalter Textil',
 u'Produktpr\xfcfer Textil',
 u'Produktveredler Textil',
 u'Pr\xe4sident',
 u'Vorpolierer Schmuck- und Kleinger\xe4teherstellung',
 u'Werkgehilfe Schmuckwarenindustrie, Taschen- und Armbanduhren'}

In [35]:
manually_retrieved2={"Aufzugsmonteur":'25132',
 u'Fachangestellter f\xfcr Medien- und Informationsdienste':'73332',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (B\xe4ckerei)':'62312',
 u'Fachverk\xe4ufer im Lebensmittelhandwerk (Fleischerei)':'62322',
 u'Film-Regisseur':'94414',
    u'Kettengoldschmied':'93522',
 u'Kraftfahrer':'51311',
 u'Masseur und medizinischer Bademeister':'81712',
 u'Online-Redakteur':'92413',
 u'Produktgestalter Textil':'28112',
 u'Produktpr\xfcfer Textil':'28122',
 u'Produktveredler Textil':'28142',
 u'Pr\xe4sident':'84394',
 u'Vorpolierer Schmuck- und Kleinger\xe4teherstellung':'24222',
 u'Werkgehilfe Schmuckwarenindustrie, Taschen- und Armbanduhren':'71382'}
for i in manually_retrieved2:
    m_bias_index[i]=manually_retrieved2[i]
len(m_bias_index),len(page_m)

(799, 812)

In [36]:
print "We do not took into account:"
set(page_m)-set(m_bias_index.keys())

We do not took into account:


{u'Angestellter',
 u'Facharbeiter',
 u'Fachkaufmann',
 u'Fachleiter',
 u'Fachpraktiker',
 u'Helfer',
 u'Kaufmann',
 u'Meister',
 u'Produktionshelfer',
 u'Sachverst\xe4ndiger',
 u'Techniker',
 u'Trainer',
 u'Verfahrensmechaniker'}

# Femenine

In [37]:
page_f=load_simple_json('de/wiki/f_bias.json')
print len(page_f)
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
f_bias_index,other=find_appropriate_prof(page_f,words,profession,profession_2,profession_2_2,profession_1,profession_3_0)

6
3 0 3 0 0 0 0


In [38]:
f_bias_index

{u'Belegentbindungspfleger': '81353',
 u'Beschlie\xdfer': '83212',
 u'Entbindungspfleger': '81353',
 u'Gleichstellungsbeauftragter': '71234',
 u'Haush\xe4lter': '83212',
 u'Pflegediensthelfer': '81301'}

# Neutral

In [39]:
page_n=load_simple_json('de/wiki/n_bias_male_labels.json')
print len(page_n)
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
n_bias_index,other=find_appropriate_prof(page_n,words,profession,profession_2,profession_2_2,profession_1,profession_3_0)

19
Audiodesigner =? Audiodesigner - Musik 94183
1 1 8 0 0 1 0


In [40]:
n_bias_index["Audiodesigner"]='94183'
n_bias_index

{u'Apotheker': '81804',
 'Audiodesigner': '94183',
 u'Bote': '51321',
 u'Bundeskanzler': '71214',
 u'Dekorationenmaler': '94612',
 u'Diakon': '83323',
 u'Direktor': '84194',
 u'Dressman': '94232',
 u'Filmgesch\xe4ftsf\xfchrer': '94494',
 u'Hilfspolizist': '53182',
 u'Host': '63401',
 u'Knecht': '11101',
 u'Medientechniker': '26313',
 u'Messebauer': '33332',
 u'M\xf6nch': '83332',
 u'Ordensbruder': '83332',
 u'Pflegedienstleiter': '81394',
 u'Projektassistent': '26313',
 u'Taxifahrer': '52112'}

In [41]:
page_n_n=load_simple_json('de/wiki/n_bias_n_labels.json')
print len(page_n_n)
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
n_bias_n_labels_index,other=find_appropriate_prof(page_n_n,words,profession,profession_2,profession_2_2,profession_1,profession_3_0)

36
Fachkraft =? Fachkraft - Trinkwasserinstallationen u. Entwässerungsanlag. 32242
Fachkraft =? Fachkraft für Metalltechnik - Zerspanungstechnik 24232
Fachkraft =? Fachkraft - Textverarbeitung für Blinde und Sehbehinderte 71432
Fachkraft =? Fachkraft - Möbel-, Küchen- und Umzugsservice 22342
Fachkraft =? Fachkraft - Solartechnik 26242
Fachkraft =? Fachkraft - Audiotechnik 94532
Fachkraft =? Fachkraft - Umweltschutz 42313
Fachkraft =? Fachkraft - Veranstaltungstechnik 94512
Fachkraft =? Fachkraft - Arbeitsförderung 83133
Fachkraft =? Fachkraft für Lasertechnik 24243
Fachkraft =? Fachkraft - Merchandising 92122
Fachkraft =? Fachkraft - landwirtschaftlicher Haushalt 83213
Fachkraft =? Beauty-Fachkraft 82322
Fachkraft =? CAD-Fachkraft - Metall 27223
Fachkraft =? CAD-Fachkraft - Bau 27223
Fachkraft =? Fachkraft im Labor 41322
Fachkraft =? Fachkraft - Abwassertechnik 34312
Fachkraft =? Hydraulik-Fachkraft 25183
Fachkraft =? Kaufmännische EDV-Fachkraft 43112
Fachkraft =? SPS-Fachkraft 43122
F

In [42]:
print "We do not took into account:"
other

We do not took into account:


[u'Fachkraft']

In [43]:
n_bias_n_labels_index

{u'Au-pair': '83111',
 u'Aufsichtsperson': '73214',
 u'Bestattungsfachkraft': '82402',
 u'B\xfcrokraft': '71402',
 u'CNC-Fachkraft': '24233',
 u'Elektrofachkraft': '26302',
 u'Elektrofachkraft f\xfcr festgelegte T\xe4tigkeiten': '26212',
 u'Elektroger\xe4tefachkraft': '26312',
 u'Fachkraft f\xfcr Abwassertechnik': '34312',
 u'Fachkraft f\xfcr Agrarservice': '11102',
 u'Fachkraft f\xfcr Arbeitssicherheit': '53123',
 u'Fachkraft f\xfcr Automatenservice': '61142',
 u'Fachkraft f\xfcr Hafenlogistik': '51332',
 u'Fachkraft f\xfcr Holz- und Bautenschutzarbeiten': '33242',
 u'Fachkraft f\xfcr Kreislauf- und Abfallwirtschaft': '34332',
 u'Fachkraft f\xfcr Kurier-, Express- und Postdienstleistungen': '51322',
 u'Fachkraft f\xfcr Lagerwirtschaft': '51312',
 u'Fachkraft f\xfcr Lebensmitteltechnik': '29202',
 u'Fachkraft f\xfcr Lederverarbeitung': '28312',
 u'Fachkraft f\xfcr Metalltechnik': '24202',
 u'Fachkraft f\xfcr M\xf6bel-, K\xfcchen- und Umzugsservice': '22342',
 u'Fachkraft f\xfcr Rohr-, 

# Assign correspondent amount of man/woman at German Labour market

In [44]:
print "We have:",len(m_bias_index)," inctances."
k=0
m_bias_prop=[]
check=[]
for i in m_bias_index:
    for beruf in subsubgroups:
        if beruf['KldB']==m_bias_index[i][:4]:
            if np.isreal(beruf[u'M\xe4nner']):
                m_bias_prop.append({"name":i,
                                "Berufsgruppe":beruf['Berufsgruppen'],
                                "KldB":beruf['KldB'],
                               "KldB5":m_bias_index[i],
                               "Frauen":float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               u'M\xe4nner':100-float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               "Overall":beruf['Ingesamt']})
                check.append(i)
            else:
                for beruf in dict_list:
                    if beruf['KldB']==m_bias_index[i][:3]:
                        if np.isreal(beruf[u'M\xe4nner']):
                            m_bias_prop.append({"name":i,
                                "Berufsgruppe":beruf['Berufsgruppen'],
                                "KldB":beruf['KldB'],
                               "KldB5":m_bias_index[i],
                               "Frauen":float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               u'M\xe4nner':100-float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               "Overall":beruf['Ingesamt']})
                            check.append(i)
                        break
            break
print len(m_bias_prop)
didnot_found=set(m_bias_index.keys())-set(check)
for i in didnot_found:
    for beruf in dict_list:
        if beruf['KldB']==m_bias_index[i][:2]:
            if np.isreal(beruf[u'M\xe4nner']):
                m_bias_prop.append({"name":i,
                                "Berufsgruppe":beruf['Berufsgruppen'],
                                "KldB":beruf['KldB'],
                               "KldB5":m_bias_index[i],
                               "Frauen":float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               u'M\xe4nner':100-float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               "Overall":beruf['Ingesamt']})
                check.append(i)
            break
didnot_found=set(m_bias_index.keys())-set(check)
print len(m_bias_prop)
print didnot_found
with open("de/m_bias_labour.json","w")as f:
    json.dump(m_bias_prop, f, indent=4) 

We have: 799  inctances.
796
799
set([])


In [45]:
print "We have:",len(f_bias_index)," inctances."
k=0
f_bias_prop=[]
check=[]
for i in f_bias_index:
    for beruf in subsubgroups:
        if beruf['KldB']==f_bias_index[i][:4]:
            if np.isreal(beruf[u'M\xe4nner']):
                f_bias_prop.append({"name":i,
                                "Berufsgruppe":beruf['Berufsgruppen'],
                                "KldB":beruf['KldB'],
                               "KldB5":f_bias_index[i],
                               "Frauen":float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               u'M\xe4nner':100-float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               "Overall":beruf['Ingesamt']})
                check.append(i)
            break
print len(f_bias_prop)
didnot_found=set(f_bias_index.keys())-set(check)
print didnot_found
with open("de/f_bias_labour.json","w")as f:
    json.dump(f_bias_prop, f, indent=4) 

We have: 6  inctances.
6
set([])


In [46]:
print "We have:",len(n_bias_index)," inctances."
k=0
n_bias_prop=[]
check=[]
for i in n_bias_index:
    for beruf in subsubgroups:
        if beruf['KldB']==n_bias_index[i][:4]:
            if np.isreal(beruf[u'M\xe4nner']):
                n_bias_prop.append({"name":i,
                                "Berufsgruppe":beruf['Berufsgruppen'],
                                "KldB":beruf['KldB'],
                               "KldB5":n_bias_index[i],
                               "Frauen":float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               u'M\xe4nner':100-float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               "Overall":beruf['Ingesamt']})
                check.append(i)
            break
print len(n_bias_prop)
didnot_found=set(n_bias_index.keys())-set(check)
print didnot_found
with open("de/n_bias_labour.json","w")as f:
    json.dump(n_bias_prop, f, indent=4) 

We have: 19  inctances.
19
set([])


In [47]:
print "We have:",len(n_bias_n_labels_index)," inctances."
k=0
n_bias_n_labels_index_prop=[]
check=[]
for i in n_bias_n_labels_index:
    for beruf in subsubgroups:
        if beruf['KldB']==n_bias_n_labels_index[i][:4]:
            if np.isreal(beruf[u'M\xe4nner']):
                n_bias_n_labels_index_prop.append({"name":i,
                                "Berufsgruppe":beruf['Berufsgruppen'],
                                "KldB":beruf['KldB'],
                               "KldB5":n_bias_n_labels_index[i],
                               "Frauen":float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               u'M\xe4nner':100-float(beruf["Frauen"]*100)/float(beruf["Ingesamt"]),
                               "Overall":beruf['Ingesamt']})
                check.append(i)
            break
print len(n_bias_n_labels_index_prop)
didnot_found=set(n_bias_n_labels_index.keys())-set(check)
print didnot_found
with open("de/n_n_bias_labour.json","w")as f:
    json.dump(n_bias_n_labels_index_prop, f, indent=4) 

We have: 35  inctances.
35
set([])


In [49]:
#Store all together

In [51]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
words=load_simple_json('de/occupation_all.json')
m_f={i:words[i][1] for i in words}#male:female

f_df=pd.DataFrame.from_dict(f_bias_prop)
n_df=pd.DataFrame.from_dict(n_bias_prop)
n_n_df=pd.DataFrame.from_dict(n_bias_n_labels_index_prop)
m_df=pd.DataFrame.from_dict(m_bias_prop)
all_together = pd.concat([n_df,n_n_df,f_df,m_df],ignore_index=True)
all_together["f_name"]=all_together['name'].map(lambda x: m_f[x] if m_f.has_key(x) else None)
#all_together["other_name"]=all_together['name'].map(lambda x: data. if m_f.has_key(x) else None)
all_together.to_csv("de/all_labour.csv", sep='\t', encoding='utf-8')
all_together#859

,Berufsgruppe,Frauen,KldB,KldB5,Männer,Overall,name,f_name
0,"Obj.Pers.Brandschutz,Arbeitssicherh(ssT)",37.700505,5318,53182,62.299495,8915,Hilfspolizist,Politesse
1,"Musik-,Gesangs-, Dirigententätigk. (ssT)",35.508637,9418,94183,64.491363,521,Audiodesigner,Audiodesignerin
2,"Aufsicht,Führung-TheaterFilm,Fernsehprod",40.187288,9449,94494,59.812712,5019,Filmgeschäftsführer,Filmgeschäftsführerin
3,Führung - Allgemeinbildende Schulen,62.997543,8419,84194,37.002457,2035,Direktor,Direktrice
4,Berufe für Post- und Zustelldienste,47.894039,5132,51321,52.105961,205963,Bote,Botin
5,Angehörige gesetzgebender Körperschaften,18.156809,7121,71214,81.843191,2908,Bundeskanzler,Bundeskanzlerin
6,Berufe in der Gemeindearbeit,58.613996,8332,83323,41.386004,10303,Diakon,Diakonin
7,"Berufe Informations,Telekommunikationst.",11.438161,2631,26313,88.561839,191884,Medientechniker,Medientechnikerin
8,"Berufe Informations,Telekommunikationst.",11.438161,2631,26313,88.561839,191884,Projektassistent,Projektassistentin
9,"Angehörige geistl. Orden, Mutterhäuser",62.571104,8333,83332,37.428896,879,Mönch,Nonne
